# File Operations

This notebook demonstrates all `Store` file operations: write, read, delete, move, copy, list, and metadata.

In [ ]:
import tempfile

from remote_store import BackendConfig, Registry, RegistryConfig, StoreProfile

tmpdir = tempfile.mkdtemp()
config = RegistryConfig(
    backends={"local": BackendConfig(type="local", options={"root": tmpdir})},
    stores={"files": StoreProfile(backend="local")},
)
registry = Registry(config)
store = registry.get_store("files")
print("Store ready.")

## Writing Files

In [ ]:
store.write("docs/readme.txt", b"Welcome to remote-store")
store.write("docs/guide.txt", b"Getting started guide")
store.write("data/report.csv", b"name,value\nalpha,1\nbeta,2")
print("Created 3 files.")

## Reading Files

In [ ]:
# Read as bytes
content = store.read_bytes("docs/readme.txt")
print(f"Bytes: {content}")

# Read as stream
stream = store.read("data/report.csv")
print(f"\nStream type: {type(stream).__name__}")
print(f"Stream content: {stream.read().decode()}")

## Listing Files and Folders

In [ ]:
print("Files in docs/:")
for f in store.list_files("docs"):
    print(f"  {f.name} ({f.size} bytes)")

print("\nSubfolders in docs/:")
for name in store.list_folders("docs"):
    print(f"  {name}/")

print("\nAll files in docs/ (recursive):")
for f in store.list_files("docs", recursive=True):
    print(f"  {f.path}")

## File and Folder Metadata

In [ ]:
info = store.get_file_info("data/report.csv")
print(f"File: {info.name}, size: {info.size}, modified: {info.modified_at}")

folder = store.get_folder_info("docs")
print(f"Folder: {folder.path}, files: {folder.file_count}, total: {folder.total_size} bytes")

## Copy and Move

In [ ]:
store.copy("docs/readme.txt", "backup/readme.txt")
print(f"Copied: exists at backup/readme.txt = {store.exists('backup/readme.txt')}")

store.move("docs/guide.txt", "archive/guide.txt")
print(f"Moved: exists at docs/guide.txt = {store.exists('docs/guide.txt')}")
print(f"Moved: exists at archive/guide.txt = {store.exists('archive/guide.txt')}")

## Delete

In [ ]:
store.delete("backup/readme.txt")
print(f"Deleted: exists = {store.exists('backup/readme.txt')}")

# delete with missing_ok
store.delete("nonexistent.txt", missing_ok=True)
print("delete(missing_ok=True) succeeded silently.")

## Cleanup

In [ ]:
import shutil

registry.close()
shutil.rmtree(tmpdir)
print("Cleaned up.")